In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import matplotlib.pyplot as plt
from scipy import spatial
from sklearn.cluster import DBSCAN

!pip install changefinder
import changefinder

!pip install lasio
import lasio
import glob
import os
import cv2
import re
from PIL import Image
from sklearn.preprocessing import MinMaxScaler

In [3]:
directory = "Features_directory" 
parent_dir = "./"
path = os.path.join(parent_dir, directory) 
os.makedirs(path, exist_ok=True) 
print("Directory '% s' created" % directory) 

directory = "Well_markers_pics" 
parent_dir = "./"
path = os.path.join(parent_dir, directory) 
os.makedirs(path, exist_ok=True) 
print("Directory '% s' created" % directory) 

directory = "Well_desc" 
parent_dir = "./"
path = os.path.join(parent_dir, directory) 
os.makedirs(path, exist_ok=True) 
print("Directory '% s' created" % directory)

directory = "Image_directory"
parent_dir = "./"
path = os.path.join(parent_dir, directory) 
os.makedirs(path, exist_ok=True) 
print("Directory '% s' created" % directory)

directory = "Org_DataFrame"
parent_dir = "./"
path = os.path.join(parent_dir, directory) 
os.makedirs(path, exist_ok=True) 
print("Directory '% s' created" % directory)

directory = "WellImages_array_directory"
parent_dir = "./"
path = os.path.join(parent_dir, directory) 
os.makedirs(path, exist_ok=True) 
print("Directory '% s' created" % directory)

directory = "WellImages_VGGFeatureVector_directory"
parent_dir = "./"
path = os.path.join(parent_dir, directory) 
os.makedirs(path, exist_ok=True) 
print("Directory '% s' created" % directory)

directory = "WellImages_EGSFeatureVector_directory"
parent_dir = "./"
path = os.path.join(parent_dir, directory) 
os.makedirs(path, exist_ok=True) 
print("Directory '% s' created" % directory)

directory = "WellImages_AutoEncoderFeatureVector_directory"
parent_dir = "./"
path = os.path.join(parent_dir, directory) 
os.makedirs(path, exist_ok=True) 
print("Directory '% s' created" % directory)

In [4]:
# import shutil
# import os
# os.remove("./FeatureVector_Matching_168_192.png")
# os.remove("./Descriptor_Matching_168_192.png")
# os.remove("./168_192_descriptor_correlation_output.csv")
# os.remove("./168_192_FeatureVector_correlation_output.csv")
# shutil.rmtree("./Well_markers_pics")
# shutil.rmtree("./Image_directory")
# shutil.rmtree("./Features_directory")
# shutil.rmtree("./Well_desc")
# shutil.rmtree("./WellImages_array_directory")
# shutil.rmtree("./WellImages_FeatureVector_directory")
# shutil.rmtree("./Org_DataFrame")

In [5]:
def findChangePoints(ts, r, order, smooth):
    '''
       r: Discounting rate
       order: AR model order
       smooth: smoothing window size T
    '''
    cf = changefinder.ChangeFinder(r=r, order=order, smooth=smooth)
    ts_score = [cf.update(p) for p in ts]
#     plt.figure(figsize=(30,4))
#     plt.plot(ts)
#     plt.figure(figsize=(30,4))
#     plt.plot(ts_score, color='red')
    return(ts_score)


def plot_change_points(ts,ts_change_loc):
    plt.figure(figsize=(30,4))
    plt.plot(ts)
    for x in ts_change_loc:
        plt.axvline(x,lw=2, color='red')
        
        
def get_change_points(var,r,order,smooth,n_largest):
    ts_score1 = findChangePoints(var, r = r, order = order, smooth = smooth)
    ts_change_loc1 = pd.Series(ts_score1).nlargest(n_largest)
    ts_change_loc1 = ts_change_loc1.index
#     plot_change_points(var,ts_change_loc1)
    return ts_change_loc1

In [6]:
from sklearn.neighbors import NearestNeighbors

def detect_and_compute(lasfile,features):
    
    df=lasio.read(lasfile)
    well_name=df.well.WELL.value
    df=df.df()
    df=df.reset_index()
    df=df[["DEPTH"]+features]
    df=df.dropna()
    
    for i in range(len(features)):
        df=df[df[features[i]]>=0]
    df=df.reset_index(drop=True)
    
    directory = "Image_directory/" + well_name
    parent_dir = "./"
    path = os.path.join(parent_dir, directory) 
    os.makedirs(path, exist_ok=True)
    
    lst=[]
    for i in features:
        idx=get_change_points(var=df[i],r=0.1,order=1,smooth=5,n_largest=40).tolist()
        lst=lst+idx
        
    model=DBSCAN(eps=32,min_samples=1)
    model.fit((np.array(lst)).reshape(len(lst),1))
    labels=model.labels_
    unq_labels=np.unique(labels).tolist()
    labels_lst=labels.tolist()
    
    depth_change_points=[]
    for i in range(len(unq_labels)):
        depth_change_points.append(np.mean(df["DEPTH"].values[np.array(lst)[np.where(labels ==unq_labels[i])[0][:].tolist()].tolist()]))
        
    depth_change_points_array=np.around(np.array(depth_change_points), decimals=2)
    depth_change_points_array=np.sort(depth_change_points_array)
    depth_change_points_array=np.delete(depth_change_points_array,[0,-1])
    
    idx_lst=[]
    dept_org=[]
    nbrs = NearestNeighbors()
    for i in range(len(depth_change_points_array)):
        temp= np.array(list(df['DEPTH'])).reshape(-1, 1)
        nbrs.fit(temp)
        distance, index = nbrs.kneighbors(np.array([[depth_change_points_array[i]]]))
        lst = distance.tolist()[0]
        lst_ = index.tolist()[0]
        dist = lst[0]
        idx = lst_[0]
        idx_lst.append(idx)
        dept_org.append(df["DEPTH"][idx])
        
#         distance,index = spatial.KDTree(df[["DEPTH"]]).query(depth_change_points_array[i])
#         idx_lst.append(index)
#         dept_org.append(df["DEPTH"][index])
    
    feature_scaled_lst=[]
    for i in range(len(features)):
        df[features[i]+"_scaled"]=(((df[features[i]]-min(df[features[i]]))/(max(df[features[i]])-min(df[features[i]])))*50)+(i*100)
        feature_scaled_lst.append(features[i]+"_scaled")
        
    feature_df=pd.DataFrame()
    feature_df["DEPTH"]=depth_change_points_array
    feature_df["dept_idx"]=idx_lst
    desc_lst=[]
    image_list = []
    for i in range(len(feature_df)):
        image_df = pd.DataFrame()
        lst=[]
        k=0
        for j in feature_scaled_lst:
            lst=lst + (df[j][feature_df["dept_idx"][i]-32:feature_df["dept_idx"][i]+32]-(k*100)).values.tolist()
            image_df[j] = (df[j][feature_df["dept_idx"][i]-32:feature_df["dept_idx"][i]+32]-(k*100)).values.tolist()
            k=k+1
        
        scaler = MinMaxScaler(feature_range=(0,255))
        scaled_keypoints_array = scaler.fit_transform(image_df)
        scaled_image_df = pd.DataFrame(scaled_keypoints_array, columns=feature_scaled_lst).astype('int')
        scaled_df_to_array = scaled_image_df.to_numpy().astype(np.uint8)
        # print(scaled_df_to_array)
        img_arr = scaled_df_to_array.reshape(8, 8, len(features))
        image_list.append(img_arr)
#         original = Image.fromarray(img_arr)
#         original.save("./Image_directory/{}/deppts_{}_image.png".format(well_name, i))
        desc_lst.append(np.around(np.array(lst),decimals=2).reshape(len(lst),))
        
    feature_df["desc"]=desc_lst
    np.save("./Well_desc/"+well_name+".npy",np.array(desc_lst))
    np.save("./WellImages_array_directory/" + well_name + "_images" + ".npy",np.array(image_list))
    
#     for imdir in sorted(os.listdir("./Image_directory")):
#         image_list = []
#         for filename in sorted(os.listdir("./Image_directory/"+ imdir), key=lambda x:int(re.findall("[0-9]+", x)[0])):
#             image = cv2.imread("./Image_directory/"+ imdir + "/" + filename)
#             image_list.append(image)
#         np.save("./WellImages_array_directory/" + imdir + "_images" + ".npy",np.array(image_list))
    
    org_df = df.copy()
    
    fig, ax = plt.subplots(figsize=(6,48))
    for i in range(len(features)):
        plt.plot(df[features[i]+"_scaled"],df["DEPTH"])
        ax.text((i*100)+25,min(df["DEPTH"])-10,features[i],size=10)

    for i in depth_change_points_array:
        plt.axhline(y=i, color='black', linestyle='-')


    plt.gca().invert_yaxis()
    plt.title(well_name)
    plt.show()
    fig.savefig('./Well_markers_pics/{}_markers.png'.format(well_name))
    feature_df.to_csv("./Features_directory/{}_features.csv".format(well_name), index=False)
    org_df.to_csv("./Org_DataFrame/{}_dataframe.csv".format(well_name), index=False)

    return org_df, feature_df, dept_org, np.array(lst)

In [7]:
data_dir="../input/ovl-las-with-labels/"
files = os.listdir(data_dir)
print(files)

In [8]:
def features(files):
    well_features = []
    for i in range(len(files)):
        features = []
        las = lasio.read("../input/ovl-las-with-labels/"+files[i])
        for curve in las.sections['Curves']:
            features.append(curve.mnemonic)
        well_features.append(features)
    
    common_features = list(set.intersection(*[set(sublist) for sublist in well_features]))
    return common_features
        
common_features = features(files)
print(common_features)

In [9]:
final_files_lst=[]
no_of_features = int(input('Enter the number of features:- '))

features_ = []
common_features = features(files)
j = 1
while j <= no_of_features:
    fea = input('Enter the name of the feature:- ')
    if fea not in common_features:
        j -= 1
    else:
        features_.append(fea)
    j += 1
        
for i in range(len(files)):
    org_df,new_df,dept,desc=detect_and_compute("../input/ovl-las-with-labels/"+files[i], features_)
    final_files_lst.append(files[i])
#     except:
#         print("missing features in ",files[i])
        
final_files_lst

In [10]:
data_dir1="./Well_desc/"
files1 = os.listdir(data_dir1)
print(files1)

In [11]:
# wellno1 = input('Enter the well number1:- ')
# wellno2 = input('Enter the well number1:- ')
# df1=pd.read_csv("./Features_directory/SES-" + wellno1 + "_features.csv")
# des1=np.load("./Well_desc/SES-" + wellno1 + ".npy")

# df2=pd.read_csv("./Features_directory/SES-" + wellno2 + "_features.csv")
# des2=np.load("./Well_desc/SES-" + wellno2 + ".npy")

In [12]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors

def output(wellno1, wellno2, method, well1_feature, well2_feature):
    
    wellno1 = wellno1
    wellno2 = wellno2
    df1=pd.read_csv("./Features_directory/SES-" + wellno1 + "_features.csv")
    des1=np.load("./Well_desc/SES-" + wellno1 + ".npy")

    df2=pd.read_csv("./Features_directory/SES-" + wellno2 + "_features.csv")
    des2=np.load("./Well_desc/SES-" + wellno2 + ".npy")

    nbrs = NearestNeighbors()

    offset= int(input('Enter the offset value:- '))

    df1_dep_lst=[]
    df2_dep_lst=[]
    dist_lst=[]

    for i in range(len(df1)):
        temp=df2[df2["DEPTH"]>(df1["DEPTH"][i]-offset)]
        temp=temp[temp["DEPTH"]<(df1["DEPTH"][i]+offset)]
        #print(temp)
        try:
            temp_min_dp=min((temp.index).to_list())
            temp_max_dp=max((temp.index).to_list())

#             if method == "Spatial KDTree Distance":
#                 temp_des=well2_feature[temp_min_dp:temp_max_dp+1]
#                 dist,idx = spatial.KDTree(temp_des).query(well1_feature[i])


#             elif method == "Cosine Similarity Score":
#                 temp = well2_feature[temp_min_dp:temp_max_dp+1]
#                 distance = cosine_similarity(well1_feature[i].reshape(1, 512), temp)
#                 lst = distance.tolist()[0]
#                 dist = max(lst)
#                 idx = lst.index(dist)

#             else:
            temp=well2_feature[temp_min_dp:temp_max_dp+1]
            nbrs.fit(temp)
            distance, index = nbrs.kneighbors(well1_feature[i].reshape(1, filters*8))
            lst = distance.tolist()[0]
            lst_ = index.tolist()[0]
            dist = lst[0]
            idx = lst_[0]

            df1_dep_lst.append(df1["DEPTH"][i])
            df2_dep_lst.append(df2["DEPTH"][idx+temp_min_dp])
            dist_lst.append(dist)
            
#             lst = distance.tolist()[0]
#             lst_ = index.tolist()[0]
#             dist = min(lst)
#             idx = lst.index(dist)
#             idx_lst.append(lst_[idx])
#             dept_org.append(df["DEPTH"][lst_[idx]])
        except:
            continue

    output_df=pd.DataFrame()
    output_df["ses-" + wellno1 + " depth"]=df1_dep_lst
    output_df["ses-" + wellno2 + " depth"]=df2_dep_lst
    output_df["distance in desc"]=dist_lst
    #print(output_df)

    val_con = pd.DataFrame()
    val_con = output_df["ses-" + wellno2 + " depth"].value_counts().to_frame()
    val_con["count"] = list(val_con["ses-" + wellno2 + " depth"])
    val_con["ses-" + wellno2 + " depth"] = list(val_con.index)
    val_con.index = range(len(val_con))
    
    final_output=pd.DataFrame()
    for i in range(len(val_con)):
        temp=output_df[output_df["ses-" + wellno2 + " depth"]==val_con["ses-" + wellno2 + " depth"][i]]
        final_output=final_output.append(temp[temp["distance in desc"]==min(temp["distance in desc"])])

    #print(final_output)
    final_output=final_output.sort_values(["ses-" + wellno1 + " depth"])
    final_output=final_output.reset_index(drop=True)

    lst=[]
    for i in range(len(final_output)):
        lst.append("H_{}".format(i))   
    final_output["Horizon"]=lst
    final_output.to_csv(wellno1 + "_" + wellno2 + "_" + method + "_" + "final_output.csv", index=False)
    
    temp1=pd.DataFrame()
    temp1["Depth"]=final_output["ses-" + wellno1 + " depth"]
    temp1["pick"]=final_output["Horizon"]
    temp1["Well name"]="ses-" + wellno1

    temp2=pd.DataFrame()
    temp2["Depth"]=final_output["ses-" + wellno2 + " depth"]
    temp2["pick"]=final_output["Horizon"]
    temp2["Well name"]="ses-" + wellno2

    structured_output=temp1.append(temp2)
    structured_output=structured_output.reset_index(drop=True)
    structured_output.to_csv(wellno1 + "_" + wellno2 + "_" + method + "_" + "structured_output.csv", index=False)
    
#     # calculating max and min depth range
#     minimum = []
#     maximum = []
#     for filename in os.listdir("./Org_DataFrame"):
#         if filename in ["SES-" + wellno1 + "_dataframe.csv", "SES-" + wellno2 + "_dataframe.csv"]:
#             org_df = pd.read_csv("./Org_DataFrame/" + filename)
#             min_depth = org_df['DEPTH'].min()
#             max_depth = org_df['DEPTH'].max()
#             minimum.append(min_depth)
#             maximum.append(max_depth)
#         else:
#             pass

#     min_depth_val = min(minimum)
#     max_depth_val = max(maximum)
    
#     org_df_well1 = pd.read_csv("./Org_DataFrame/SES-" + wellno1 + "_dataframe.csv")
#     org_df_well2 = pd.read_csv("./Org_DataFrame/SES-" + wellno2 + "_dataframe.csv")
    
#     fig = plt.figure(1, figsize=(15,48))

#     ax1 = fig.add_subplot(121)
#     ax2 = fig.add_subplot(122)

#     features = ['GR', 'DTC', 'RHOB']
#     depth_change_points_array = list(structured_output[structured_output["Well name"] == "ses-" + wellno1]["Depth"])

#     for i in range(len(features)):
#         ax1.plot(org_df_well1[features[i]+"_scaled"],org_df_well1["DEPTH"])
#         ax1.text((i*100)+25,min(org_df_well1["DEPTH"])-10,features[i],size=10)

#     for i in depth_change_points_array:
#         ax1.axhline(y=i, color='black', linestyle='-')

#     ax1.invert_yaxis()
#     ax1.set(title="Well SES-" + wellno1)
#     ax1.set_ylim(max_depth_val,min_depth_val)
#     #ax1.set_ylim(max_depth_val,3850)

#     ax1.set_yticks(np.arange(min_depth_val,max_depth_val,50))
#     #ax1.set_yticks(np.arange(3850,max_depth_val,50))

#     features = ['GR', 'DTC', 'RHOB']
#     depth_change_points_array = list(structured_output[structured_output['Well name'] == "ses-" + wellno2]['Depth'])

#     for i in range(len(features)):
#         ax2.plot(org_df_well2[features[i]+"_scaled"],org_df_well2["DEPTH"])
#         ax2.text((i*100)+25,min(org_df_well2["DEPTH"])-10,features[i],size=10)

#     for i in depth_change_points_array:
#         ax2.axhline(y=i, color='black', linestyle='-')


#     ax2.invert_yaxis()
#     ax2.set(title="Well SES-" + wellno2)
#     ax2.set_ylim(max_depth_val,min_depth_val)
#     #ax2.set_ylim(max_depth_val,3850)


#     plt.setp(ax2.get_yticklabels(), visible = False)
#     plt.subplots_adjust(wspace = 0)

#     plt.show()
    
    return output_df, val_con, final_output, structured_output
    

# Wells Images Array and Display Images

In [13]:
# img_arr_well1 = np.load("./WellImages_array_directory/SES-" + wellno1 + "_images.npy")
# print(img_arr_well1)

# img_arr_well2 = np.load("./WellImages_array_directory/SES-" + wellno2 + "_images.npy")
# print(img_arr_well2)

In [14]:
# data_dir="./Image_directory/SES-" + wellno1
# files = os.listdir(data_dir)

# for i in range(len(files)):
#     image = cv2.imread("./Image_directory/SES-" + wellno1 + "/" + files[i])
#     #print(image.shape)
#     plt.imshow(image)
#     plt.show()

In [15]:
# data_dir="./Image_directory/SES-" + wellno2
# files = os.listdir(data_dir)

# for i in range(len(files)):
#     image = cv2.imread("./Image_directory/SES-" + wellno2 + "/" + files[i])
#     #print(image.shape)
#     plt.imshow(image)
#     plt.show()

## Descriptor_Features

In [16]:
# des1

In [17]:
# des2

## VGG_Features

In [18]:
# from keras.applications.resnet import ResNet50
# from keras.preprocessing import image
# from keras.applications.resnet import preprocess_input
# import numpy as np
# from PIL import Image as pil_image
# from keras.applications.vgg16 import VGG16
# from keras.applications.vgg16 import preprocess_input


# model = VGG16(weights='imagenet', include_top=False, input_shape=(32,32,3))
# for layer in model.layers:
#     layer.trainable=False
# print(model.summary())

# for direc in sorted(os.listdir("./Image_directory")):
#     i = 0
#     for img_path in sorted(os.listdir("./Image_directory/"+ direc), key=lambda x:int(re.findall("[0-9]+", x)[0])):
#         image_no = len(os.listdir("./Image_directory/"+ direc))
#         img = image.load_img("./Image_directory/"+ direc + "/" + img_path, target_size=(32, 32))
#         x = image.img_to_array(img)
#         x = np.expand_dims(x, axis=0)
#         x = preprocess_input(x)
#         #predict image features
#         images_features = model.predict(x)
#         vector = images_features[0]
#         feature = np.empty(shape=(image_no,512))
#         feature[i] = vector
#         i += 1
#     feature = np.round(feature,2)
#     np.save("./WellImages_VGGFeatureVector_directory/" + direc + "_VGG_feature_vector" + ".npy",np.array(feature))

In [19]:
# VGG_feavec_well1 = np.load("./WellImages_VGGFeatureVector_directory/SES-" + wellno1 + "_VGG_feature_vector.npy")
# print(VGG_feavec_well1)
# print('\r')
# VGG_feavec_well2 = np.load("./WellImages_VGGFeatureVector_directory/SES-" + wellno2 + "_VGG_feature_vector.npy")
# print(VGG_feavec_well2)

## Entropy_Gaussian_Sobel_Features

In [20]:
# import cv2
# import pandas as pd
# from skimage.filters.rank import entropy
# from skimage.morphology import disk
# from scipy import ndimage as nd
# from skimage.filters import sobel

# #files = sorted(os.listdir("./Image_directory/"+ "SES-" + wellno1), key=lambda x:int(re.findall("[0-9]+", x)[0]))


# for imdir in sorted(os.listdir("./Image_directory")):
#     lst = []
#     for filename in sorted(os.listdir("./Image_directory/"+ imdir), key=lambda x:int(re.findall("[0-9]+", x)[0])):
#         img = cv2.imread("./Image_directory/"+ imdir + "/" + filename)

#         df = pd.DataFrame()

#         img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#         img2 = img.reshape(-1)
#         df['Original Image'] = img2

#         entropy_img = entropy(img, disk(1))
#         entropy1 = entropy_img.reshape(-1)
#         df['Entropy'] = entropy1

#         gaussian_img = nd.gaussian_filter(img, sigma=3)
#         gaussian_img1 = gaussian_img.reshape(-1)
#         df['Gaussian s3'] = gaussian_img1

#         sobel_img = sobel(img)
#         sobel1 = sobel_img.reshape(-1)
#         df['Sobel'] = sobel1

#         x = list(df['Entropy']) + list(df['Gaussian s3']) + list(df['Sobel'])
#         lst.append(x)
#     np.save("./WellImages_EGSFeatureVector_directory/" + imdir + "_EGS_feature_vector" + ".npy",np.array(lst))

In [21]:
# EGS_feavec_well1 = np.load("./WellImages_EGSFeatureVector_directory/SES-" + wellno1 + "_EGS_feature_vector.npy")
# print(EGS_feavec_well1)
# print('\r')
# EGS_feavec_well2 = np.load("./WellImages_EGSFeatureVector_directory/SES-" + wellno2 + "_EGS_feature_vector.npy")
# print(EGS_feavec_well2)

## AutoEncoder_Features

In [22]:
X_1 = np.load('./WellImages_array_directory/SES-168_images.npy')
X_2 = np.load('./WellImages_array_directory/SES-178_images.npy')
X_3 = np.load('./WellImages_array_directory/SES-181_images.npy')
X_4 = np.load('./WellImages_array_directory/SES-182_images.npy')
X_5 = np.load('./WellImages_array_directory/SES-184_images.npy')
X_6 = np.load('./WellImages_array_directory/SES-189_images.npy')
X_7 = np.load('./WellImages_array_directory/SES-191_images.npy')
X_8 = np.load('./WellImages_array_directory/SES-192_images.npy')
X_9 = np.load('./WellImages_array_directory/SES-193_images.npy')

x = np.concatenate([X_1, X_2, X_3, X_4, X_5, X_6, X_7, X_8, X_9], axis=0)

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(x, test_size=0.2, random_state=42)

In [24]:
import tensorflow

from keras.layers import Dense, Flatten, Reshape, Input, InputLayer
from keras.models import Sequential, Model

def build_autoencoder(img_shape, filters):
    # The encoder
    encoder_input = tensorflow.keras.layers.Input(shape=img_shape, name="encoder_input")

    encoder_conv_layer1 = tensorflow.keras.layers.Conv2D(filters=filters, kernel_size=(3, 3), padding="same", strides=1, activation='leaky_relu', name="encoder_conv_1")(encoder_input)
    # Pooling Layer #1
    #pool1 = tensorflow.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2, name='pool1')(encoder_conv_layer1)
    #encoder_norm_layer1 = tensorflow.keras.layers.BatchNormalization(name="encoder_norm_1")(encoder_conv_layer1)
    #encoder_activ_layer1 = tensorflow.keras.layers.LeakyReLU(name="encoder_leakyrelu_1")(encoder_conv_layer1)

    encoder_conv_layer2 = tensorflow.keras.layers.Conv2D(filters=filters*2, kernel_size=(3,3), padding="same", strides=2, activation='leaky_relu', name="encoder_conv_2")(encoder_conv_layer1)
    #encoder_norm_layer2 = tensorflow.keras.layers.BatchNormalization(name="encoder_norm_2")(encoder_conv_layer2)
    #encoder_activ_layer2 = tensorflow.keras.layers.LeakyReLU(name="encoder_activ_layer_2")(encoder_conv_layer2)

    encoder_conv_layer3 = tensorflow.keras.layers.Conv2D(filters=filters*4, kernel_size=(3,3), padding="same", strides=2, activation='leaky_relu', name="encoder_conv_3")(encoder_conv_layer2)
    #encoder_norm_layer3 = tensorflow.keras.layers.BatchNormalization(name="encoder_norm_3")(encoder_conv_layer3)
    #encoder_activ_layer3 = tensorflow.keras.layers.LeakyReLU(name="encoder_activ_layer_3")(encoder_conv_layer3)

    encoder_conv_layer4 = tensorflow.keras.layers.Conv2D(filters=filters*8, kernel_size=(3,3), padding="same", strides=2, activation='leaky_relu', name="encoder_conv_4")(encoder_conv_layer3)
    #encoder_norm_layer4 = tensorflow.keras.layers.BatchNormalization(name="encoder_norm_4")(encoder_conv_layer4)
    #encoder_activ_layer4 = tensorflow.keras.layers.LeakyReLU(name="encoder_activ_layer_4")(encoder_conv_layer4)

    encoder = tensorflow.keras.models.Model(encoder_input, encoder_conv_layer4, name="encoder_model")

    # The decoder
    decoder_input = tensorflow.keras.layers.Input(shape=(1, 1, filters*8), name="decoder_input")

    decoder_conv_tran_layer1 = tensorflow.keras.layers.Conv2DTranspose(filters=filters*4, kernel_size=(3, 3), padding="same", strides=2, activation='leaky_relu', name="decoder_conv_tran_1")(decoder_input)
    #decoder_norm_layer1 = tensorflow.keras.layers.BatchNormalization(name="decoder_norm_1")(decoder_conv_tran_layer1)
    #decoder_activ_layer1 = tensorflow.keras.layers.LeakyReLU(name="decoder_leakyrelu_1")(decoder_conv_tran_layer1)

    decoder_conv_tran_layer2 = tensorflow.keras.layers.Conv2DTranspose(filters=filters*2, kernel_size=(3, 3), padding="same", strides=2, activation='leaky_relu', name="decoder_conv_tran_2")(decoder_conv_tran_layer1)
    #decoder_norm_layer2 = tensorflow.keras.layers.BatchNormalization(name="decoder_norm_2")(decoder_conv_tran_layer2)
    #decoder_activ_layer2 = tensorflow.keras.layers.LeakyReLU(name="decoder_leakyrelu_2")(decoder_conv_tran_layer2)

    decoder_conv_tran_layer3 = tensorflow.keras.layers.Conv2DTranspose(filters=filters, kernel_size=(3, 3), padding="same", strides=2, activation='leaky_relu', name="decoder_conv_tran_3")(decoder_conv_tran_layer2)
    #decoder_norm_layer3 = tensorflow.keras.layers.BatchNormalization(name="decoder_norm_3")(decoder_conv_tran_layer3)
    #decoder_activ_layer3 = tensorflow.keras.layers.LeakyReLU(name="decoder_leakyrelu_3")(decoder_conv_tran_layer3)

    decoder_conv_tran_layer4 = tensorflow.keras.layers.Conv2DTranspose(filters=no_of_features, kernel_size=(3, 3), padding="same", strides=1, activation='leaky_relu', name="decoder_conv_tran_4")(decoder_conv_tran_layer3)
    #decoder_norm_layer4 = tensorflow.keras.layers.BatchNormalization(name="decoder_norm_4")(decoder_conv_tran_layer4)
    #decoder_activ_layer4 = tensorflow.keras.layers.LeakyReLU(name="decoder_leakyrelu_4")(decoder_conv_tran_layer4)

#     decoder_conv_tran_layer5 = tensorflow.keras.layers.Conv2DTranspose(filters=3, kernel_size=(3, 3), padding="same", strides=1, activation='leaky_relu', name="decoder_conv_tran_5")(decoder_conv_tran_layer4)
#     #decoder_norm_layer5 = tensorflow.keras.layers.BatchNormalization(name="decoder_norm_5")(decoder_conv_tran_layer5)
#     decoder_activ_layer5 = tensorflow.keras.layers.LeakyReLU(name="decoder_leakyrelu_5")(decoder_conv_tran_layer5)

    decoder = tensorflow.keras.models.Model(decoder_input, decoder_conv_tran_layer4, name="decoder_model")
    
    return encoder, decoder

In [25]:
#filters= int(input('Enter the value of filter:- '))
filters= int(input('Enter the number of filters:- '))

IMG_SHAPE = (8, 8, no_of_features)
encoder, decoder = build_autoencoder(IMG_SHAPE, filters)

In [26]:
encoder.summary()

In [27]:
decoder.summary()

In [28]:
inp = Input(IMG_SHAPE)
code = encoder(inp)
reconstruction = decoder(code)

autoencoder = Model(inp,reconstruction)
autoencoder.compile(optimizer='adamax', loss='mse', metrics=['accuracy'])

history = autoencoder.fit(X_train, X_train, epochs=100, batch_size=32, shuffle=True, validation_data=(X_test, X_test))

In [29]:
autoencoder.summary()

In [30]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [31]:
encoded_data = encoder.predict(X_test)
decoded_data = decoder.predict(encoded_data)

In [32]:
# wellno1 = "182"
# wellno2 = "168"

# img_arr_well1 = np.load("./WellImages_array_directory/SES-" + wellno1 + "_images.npy")
# encoded_data = encoder.predict(img_arr_well1)

# Autoencoder_feavec_well1 = encoded_data.reshape(encoded_data.shape[0], 512)
# print(Autoencoder_feavec_well1)

# img_arr_well2 = np.load("./WellImages_array_directory/SES-" + wellno2 + "_images.npy")
# encoded_data = encoder.predict(img_arr_well2)

# Autoencoder_feavec_well2 = encoded_data.reshape(encoded_data.shape[0], 512)
# print(Autoencoder_feavec_well2)

In [33]:
# from sklearn.preprocessing import MinMaxScaler

# scaler = MinMaxScaler()
# model1=scaler.fit(Autoencoder_feavec_well1)
# Autoencoder_feavec_well1=model1.transform(Autoencoder_feavec_well1)

# model2=scaler.fit(Autoencoder_feavec_well2)
# Autoencoder_feavec_well2=model2.transform(Autoencoder_feavec_well2)

# **Result**

In [34]:
sequence = input('Enter the well pair sequence with comma:- ').split()
First_pair = sequence[:-1]
Second_pair = sequence[1:]

well_pairs = list(zip(First_pair, Second_pair))

In [35]:
# First_pair = ["181", "189", "193", "178", "191", "184", "168", "192"]
# Second_pair = ["189", "193", "178", "191", "184", "168", "192", "182"]

# well_pairs = zip(First_pair, Second_pair)

In [36]:
"""
Descriptor_Features and EGS_Features can not work with Cosine Similarity Score and Nearest Neighbours
because need reshaping from 192 to 512
"""

"""
VGG_Features giving error because Feature Input contains NaN, infinity or a value too large for dtype('float64')
except in case of Spatial KDTree Distance Method but repeating ses-192 depth values because of same distance
"""

# Spatial KDTree Distance
# output_df, val_con, final_output, structured_output = output("168", "192", "Spatial KDTree Distance", des1, des2)
# output_df, val_con, final_output, structured_output = output("168", "192", "Spatial KDTree Distance", VGG_feavec_well1, VGG_feavec_well2)
# output_df, val_con, final_output, structured_output = output("168", "192", "Spatial KDTree Distance", EGS_feavec_well1, EGS_feavec_well2)
# output_df, val_con, final_output, structured_output = output("168", "192", "Spatial KDTree Distance", Autoencoder_feavec_well1, Autoencoder_feavec_well2)


# Cosine Similarity Score
# output_df, val_con, final_output, structured_output = output("168", "192", "Cosine Similarity Score", Autoencoder_feavec_well1, Autoencoder_feavec_well2)


# Nearest Neighbours

from sklearn.preprocessing import MinMaxScaler

for wellno1, wellno2 in well_pairs:
    # print("{}-{}".format(wellno1, wellno2))
    img_arr_well1 = np.load("./WellImages_array_directory/SES-" + wellno1 + "_images.npy")
    Autoencoder_feavec_well1 = encoder.predict(img_arr_well1).reshape(img_arr_well1.shape[0], filters*8)

    img_arr_well2 = np.load("./WellImages_array_directory/SES-" + wellno2 + "_images.npy")
    Autoencoder_feavec_well2 = encoder.predict(img_arr_well2).reshape(img_arr_well2.shape[0], filters*8)
     
    scaler = MinMaxScaler()
    model1=scaler.fit(Autoencoder_feavec_well1)
    Autoencoder_feavec_well1=model1.transform(Autoencoder_feavec_well1)
    #print(Autoencoder_feavec_well1)

    model2=scaler.fit(Autoencoder_feavec_well2)
    Autoencoder_feavec_well2=model2.transform(Autoencoder_feavec_well2)
    #print(Autoencoder_feavec_well2)

    output_df, val_con, final_output, structured_output = output(wellno1, wellno2, "Nearest Neighbours", Autoencoder_feavec_well1, Autoencoder_feavec_well2)
    #print(final_output)
    
# output_df, val_con, final_output, structured_output = output(wellno1, wellno2, "Nearest Neighbours", Autoencoder_feavec_well1, Autoencoder_feavec_well2)

In [37]:
df1 = pd.read_csv('./181_189_Nearest Neighbours_final_output.csv')
df2 = pd.read_csv('./189_193_Nearest Neighbours_final_output.csv')
df3 = pd.read_csv('./193_178_Nearest Neighbours_final_output.csv')
df4 = pd.read_csv('./178_191_Nearest Neighbours_final_output.csv')
df5 = pd.read_csv('./191_184_Nearest Neighbours_final_output.csv')
df6 = pd.read_csv('./184_168_Nearest Neighbours_final_output.csv')
df7 = pd.read_csv('./168_192_Nearest Neighbours_final_output.csv')
df8 = pd.read_csv('./192_182_Nearest Neighbours_final_output.csv')

In [38]:
# def update_df(df1, df2):
#     common_col = list(df1.columns)[1]
#     horizon = list(df2['Horizon'])
#     j = 1
#     for i in range(len(df2)):
#         if list(df2[common_col])[i] not in list(df1[common_col]):
#             horizon[i] = 'H_' + str(int(sorted(list(df1['Horizon']), key=lambda x: int(x[2:]))[-1][2:]) + j)
#             j += 1
#         else:
#             idx = list(df1[common_col]).index(list(df2[common_col])[i])
#             horizon[i] = df1.loc[idx, 'Horizon']

#     df2['Horizon'] = horizon
#     return df2

In [39]:
# df1

In [40]:
# final_output = update_df(df1, df2)
# # final_output = update_df(df2, df3)
# # final_output = update_df(df3, df4)
# # final_output = update_df(df4, df5)
# # final_output = update_df(df5, df6)
# # final_output = update_df(df6, df7)
# # final_output = update_df(df7, df8)

# temp1=pd.DataFrame()
# temp1["Depth"]=final_output[list(final_output.columns)[0]]
# temp1["pick"]=final_output["Horizon"]
# temp1["Well name"]=list(final_output.columns)[0][:-6]

# temp2=pd.DataFrame()
# temp2["Depth"]=final_output[list(final_output.columns)[1]]
# temp2["pick"]=final_output["Horizon"]
# temp2["Well name"]=list(final_output.columns)[1][:-6]

# structured_output=temp1.append(temp2)
# structured_output=structured_output.reset_index(drop=True)
# #structured_output.to_csv("168_192_FeatureVector_correlation_output.csv")
# structured_output

In [41]:
def update_df():
#     First_pair = ["181", "189", "193", "178", "191", "184", "168", "192"]
#     Second_pair = ["189", "193", "178", "191", "184", "168", "192", "182"]

#     well_pairs = list(zip(First_pair, Second_pair))
    #df = pd.read_csv('./' + well_pairs[0][0] + "_" + well_pairs[0][1] + '_Nearest Neighbours_structured_output.csv')
    #df.to_csv(well_pairs[0][0] + "_" + well_pairs[0][1] + "_updated_structured_output.csv", index=False)
    well_df = pd.DataFrame()
    
    for i in range(len(well_pairs)):
        try:
            df = pd.read_csv('./' + well_pairs[i][0] + "_" + well_pairs[i][1] + '_Nearest Neighbours_final_output.csv')
            #print(df)
            
            temp1=pd.DataFrame()
            temp1["Depth"]=df[list(df.columns)[0]]
            temp1["pick"]=df["Horizon"]
            temp1["Well name"]=list(df.columns)[0][:-6]

            temp2=pd.DataFrame()
            temp2["Depth"]=df[list(df.columns)[1]]
            temp2["pick"]=df["Horizon"]
            temp2["Well name"]=list(df.columns)[1][:-6]

            structured_output=temp1.append(temp2)
            structured_output=structured_output.reset_index(drop=True)
            well_df = well_df.append(structured_output)
            
            df_ = pd.read_csv('./' + well_pairs[i+1][0] + "_" + well_pairs[i+1][1] + '_Nearest Neighbours_final_output.csv')

            common_col = list(df.columns)[1]
            horizon = list(df_['Horizon'])
            j = 1
            for k in range(len(df_)):
                if list(df_[common_col])[k] not in list(df[common_col]):
                    horizon[k] = 'H_' + str(int(sorted(list(df['Horizon']), key=lambda x: int(x[2:]))[-1][2:]) + j)
                    j += 1
                else:
                    idx = list(df[common_col]).index(list(df_[common_col])[k])
                    horizon[k] = df.loc[idx, 'Horizon']

            df_['Horizon'] = horizon
            df_.to_csv(well_pairs[i+1][0] + "_" + well_pairs[i+1][1] + '_Nearest Neighbours_final_output.csv', index=False)
            
            
            #structured_output.to_csv(well_pairs[i+1][0] + "_" + well_pairs[i+1][1] + "_updated_structured_output.csv", index=False)
        
        except:
            pass
    
    well_df=well_df.reset_index(drop=True)
    well_df.drop_duplicates(inplace=True)
    return well_df
        
output = update_df()
output.to_csv('final_output.csv', index=False)

In [42]:
output

In [43]:
output['pick'].value_counts()   #.to_frame()#.loc['H_0', 'pick']